In [ ]:
!pip install openai -q
!pip install backoff
!pip install tenacity

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
# drive.mount("/gdrive", force_remount=True)

In [ ]:
import openai
import pandas as pd
import numpy as np
from getpass import getpass

openai.api_key = getpass()

In [ ]:
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity
COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDINGS_MODEL = "text-embedding-ada-002"

In [ ]:
ls

In [ ]:
%cd 'Patents and software'
# ls

In [ ]:
patent_df = pd.read_csv('patent_three (4).csv')
software_df = pd.read_csv('software_catalog (2).csv')
# patent_df

In [ ]:
patent_df

In [ ]:
technology_info_list = ['technology area', 'Technology area text', 'Patent name', 'Purpose', 'overview',	'Technology',	'benefits',	'Applications',	'Technology Details']
software_info_list = ['software area', 'area', 'software name', 'overview']

In [ ]:
software_file_name = 'software name_embeddings.csv'
technology_file_name = 'technology_embeddings.csv'

In [ ]:
class database:

    def __init__(self, df, info_list, name, search_term_vector, no_to_get, file_name):
        self.df = df
        self.info_list = info_list
        self.col_name = name
        self.col_to_get = 'details_to_convert'
        self.search_term_vector = search_term_vector
        self.no_to_get = no_to_get
        self.file_name = file_name

    def func_get_details(self):
          for i in range(len(self.info_list)):
            if i == 0:
                self.df[self.col_to_get] = self.df[self.info_list[i]]
            else:
                self.df[self.col_to_get] += ' ' + self.df[self.info_list[i]]

    def func_get_embeddings(self):
        # patent_technology_df
        self.df['embedding'] = self.df[self.col_to_get].apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))
        self.df.to_csv(self.col_name+'_embeddings.csv')

    def func_load_embeddings_from_file_name(self):
        self.df = pd.read_csv(self.file_name)

    def func_load_embeddings(self):
        self.df = pd.read_csv(self.col_name+'_embeddings.csv')

    def func_get_sim(self):
        self.df["similarities"] = self.df['embedding'].apply(lambda x: cosine_similarity(x, search_term_vector))
        self.df_sorted = self.df.sort_values("similarities", ascending=False).head(20)
        self.df_sorted.to_csv(self.col_name+'_sorted.csv')

    def func_get_feed(self):
        self.df_feed = self.df.iloc[0:self.no_to_get][self.col_name].values

    def func_get_setup(self):
        self.func_get_details()
        self.func_get_embeddings()
        self.func_get_sim()
        self.func_get_feed()

    def func_load_setup(self):
        self.func_get_details()
        self.func_load_embeddings()
        self.func_get_sim()
        self.func_get_feed()

    def func_query(self):
        self.func_get_sim()
        self.func_get_feed()


In [ ]:
def get_keywords(text):
    return openai.Completion.create(
        prompt='Extract keywords from the following text and output as a csv file: '+text,
        temperature=1,
        max_tokens=500,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        model=COMPLETIONS_MODEL
    )["choices"][0]["text"].strip(" \n")

In [ ]:
style = input('Enter a style: ')
type_of_English = input('Enter a type of English: ')
question = input('Enter a question: ')

keywords = get_keywords(question)

# semantic search
search_term_vector = get_embedding(keywords, engine="text-embedding-ada-002")
search_term_vector

In [ ]:
technology_embeddings_new = pd.read_csv('technology_embeddings2.csv')

In [ ]:
technology_embeddings_new['embedding'].values.shape

In [ ]:
software_obj.func_get_sim()

In [ ]:
software_obj = database(software_df, software_info_list, 'software name', search_term_vector, no_to_get, software_file_name)
software_obj.func_load_setup()

In [ ]:
type(search_term_vector)

In [ ]:
search_term_vector = keywords
no_to_get = 2

patent_obj = database(patent_df, technology_info_list, 'technology', search_term_vector, no_to_get)
software_obj = database(software_df, software_info_list, 'software name', search_term_vector, no_to_get)
patent_obj.func_get_setup()
software_obj.func_get_setup()

In [ ]:
# print(patent.df_feed)

context = ''
for i in patent.df_feed:
    context += ', ' + i
for i in software.df_feed:
    context += ', ' + i

print(context)

In [ ]:
# style = input('Enter a style: ')
# type_of_English = input('Enter a type of English':)
# question = input('Enter a question: ')

In [ ]:
prompt = f"""Answer the following question using only the context below. Answer in the style of {style} in {type_of_English}. If you don't know the answer for certain, say I don't know. 

Provide the patent number, author and location of invention with each technology used.

Each comma in the context separates each technology.

Context:
{context}

Q: {question}
A:"""

openai.Completion.create(
    prompt=prompt,
    temperature=1,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")